In [ ]:
from src.render.mesh_renderer import MeshPointsRenderer   

from src.config import get_parser

config = get_parser().parse_args(args=[])   

config.log_batch_interval = 100
config.fast_outline_size =  128
config.fast_image_size = 128
config.fast_batch_size = 4
config.raster_faces_per_pixel = 4
